In [1]:
import pandas as pd
import numpy as np
import random
import os
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\tensorf

#### Data with random zeros

In [2]:
nn_df = pd.read_csv('data/dataset_10000.csv')

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tensorflow.set_random_seed(SEED)
#tensorflow.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

#### The Data

In [5]:
df = pd.read_csv('data/dataset_10000.csv')
class_dict = {'A':0, 'B':1, 'C':2}
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7000, 3), (3000, 3), (7000,), (3000,))

In [6]:
X_train[:2], X_test[:2]

(array([[ 0.,  0.,  0.],
        [ 0.,  4., 34.]]),
 array([[18., 12.,  0.],
        [ 7.,  0., 13.]]))

#### The Environment

In [7]:
import copy
from gym import Env
from gym.spaces import Discrete, Box

In [8]:
class SyntheticEnv(Env):
    def __init__(self, X, Y, random=True):
        super(SyntheticEnv, self).__init__()
        self.action_space = Discrete(6)
        self.observation_space = Box(0, 1.5, (3,))
        self.actions = ['A', 'B', 'C', 'length', 'width', 'height']
        self.max_steps = 7
        self.X = X
        self.Y = Y
        self.sample_num = len(X)
        self.idx = -1
        self.x = np.zeros((3,), dtype=np.float32)
        self.y = np.nan
        self.state = np.zeros((3,), dtype=np.float32)
        self.num_classes = 3
        self.episode_length = 0
        self.trajectory = []
        self.total_reward = 0
        self.random = random
        
    
    def step(self, action):
        #print('A step in the environment')
        #print(f'action: {action}')
        self.episode_length += 1
        reward = 0
        if self.episode_length == self.max_steps: # episode too long
            #print('Reached max steps')
            reward -=1
            self.total_reward -=1
            terminated = True
            done = True
            y_actual = self.y
            y_pred = np.nan
            is_success = False
        elif action < self.num_classes: #diagnosis (terminal action)
            #print('Terminal action')
            if action == self.y:
                reward +=1
                self.total_reward += 1
                is_success = True
            else:
                reward -= 1
                self.total_reward -= 1
                is_success = False
            terminated = False
            done = True
            y_actual = self.y
            y_pred = action
        elif self.actions[action] in self.trajectory: #action already picked 
            #print('Repeated action')
            terminated = False
            reward -= 1
            self.total_reward -= 1
            done = False
            y_actual = np.nan
            y_pred = np.nan
            is_success = None
        else: #new feature being acquired
            #print('Acquiring new feature')
            terminated = False
            reward += 1
            self.total_reward += 1
            done = False
            self.state = self.get_next_state(action-self.num_classes)
            y_actual = np.nan
            y_pred = np.nan
            is_success = None
        self.trajectory.append(self.actions[action])
        info = {'index': self.idx, 'episode_length':self.episode_length, 'reward': self.total_reward, 'y_pred': y_pred, 
                'y_actual': y_actual, 'trajectory':self.trajectory, 'terminated':terminated, 'is_success': is_success}
        #self.render()
        return self.state, reward, done, info
            
    
    def render(self):
        print(f'STEP {self.episode_length} for index {self.idx}')
        print(f'x: {self.x}')
        print(f'y: {self.y}')
        print(f'Current state: {self.state}')
        print(f'Total reward: {self.total_reward}')
        print(f'Trajectory: {self.trajectory}')
        
            
    
    def reset(self):
        #print('RESETTING THE ENVIRONMENT')
        if self.random:
            self.idx = random.randint(0, self.sample_num-1)
        else:
            self.idx += 1
            if self.idx == len(self.X):
                raise StopIteration()
        #print(f'New idx: {self.idx}')
        self.x, self.y = self.X[self.idx], self.Y[self.idx]
        #print(f'New x: {self.x}')
        #print(f'New y: {self.y}')
        self.state = np.zeros((3,), dtype=np.float32)
        #print(f'New state: {self.state}')
        self.trajectory = []
        #print(f'New trajectory: {self.trajectory}')
        self.episode_length = 0
        #print(f'New episode length: {self.episode_length}')
        self.total_reward = 0
        #print(f'New total reward: {self.total_reward}')
        return self.state
        
    
    def get_next_state(self, feature_idx):
        self.x = self.x.reshape(-1, 3)
        x_value = self.x[0, feature_idx]
        next_state = copy.deepcopy(self.state)
        next_state[feature_idx] = x_value
        return next_state

In [9]:
training_env = SyntheticEnv(X_train, y_train)

#### The Agent

In [10]:
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from stable_baselines import bench, logger

C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [11]:
def stable_dqn():
    env = SyntheticEnv(X_train, y_train)
    env = bench.Monitor(env, logger.get_dir())
    model = DQN('MlpPolicy', training_env, verbose=1, seed=SEED, n_cpu_tf_sess=1)
    model.learn(total_timesteps=int(1.2e5), log_interval=10000)
    #model.save('models/synthetic_stable_dqn.pkl')
    env.close()
    return model

dqn_model = stable_dqn()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 10000    |
| mean 100 episode reward | -0       |
| steps                   | 20588    |
| success rate            | 0.48     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 20000    |
| mean 100 episode reward | 0.1      |
| steps                   | 31344    |
| success rate            | 0.56     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 30000    |
| mean 100 episode reward | -0.1     |
| steps                   | 41686    |
| success rate            | 0.46     |
--------------------------------------


#### Performance Evaluation

In [12]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve

In [13]:
def multiclass(actual_class, pred_class, average = 'macro'):

    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        other_class = [x for x in unique_class if x != per_class]
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc
    avg = sum(roc_auc_dict.values()) / len(roc_auc_dict)
    return avg

In [14]:
def test(ytest, ypred):
    acc = accuracy_score(ytest, ypred)
    f1 = f1_score(ytest, ypred, average ='macro', labels=np.unique(ytest))
    try:
        roc_auc = multiclass(ytest, ypred)
    except:
        roc_auc = None
    return acc, f1, roc_auc

In [15]:
def get_avg_length_reward(df):
    length = np.mean(df.episode_length)
    reward = np.mean(df.reward)
    return length, reward

In [16]:
def synthetic_dqn_eval(dqn_model):
    attempts, correct = 0,0
    test_df = pd.DataFrame()

    env = SyntheticEnv(X_test, y_test, random=False)

    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                action, _states = dqn_model.predict(obs, deterministic=True)
                obs, rew, done,info = env.step(action)
                #if (done==True) & (np.isfinite(info['y_pred'])):
                if done == True:
                    test_df = test_df.append(info, ignore_index=True)
                #print('....................TEST DF ....................')
                #if len(test_df) != 0:
                #    print(test_df.head())

    except StopIteration:
        print('Testing done.....')
    return test_df

test_df = synthetic_dqn_eval(dqn_model)

Testing done.....


In [17]:
len(X_test), len(test_df)

(3000, 3000)

In [18]:
y_pred_df = test_df[test_df['y_pred'].notna()]
success_df = y_pred_df[y_pred_df['y_pred']== y_pred_df['y_actual']]
len(success_df)

1403

In [19]:
success_rate = len(success_df)/len(test_df)*100
success_rate

46.766666666666666

In [20]:
#avg length and return 
avg_length, avg_return = get_avg_length_reward(test_df)
avg_length, avg_return

(1.0, -0.06466666666666666)

In [21]:
acc, f1, roc_auc = test(y_pred_df['y_actual'], y_pred_df['y_pred'])
acc, f1, roc_auc

(0.4676666666666667, 0.21243091831327124, 0.5)

#### Failing episodes

In [22]:
#episodes that reach max_steps and are terminated forcefully
na_df = test_df[test_df['y_pred'].isna()]
na_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred


In [23]:
na_df.index

Int64Index([], dtype='int64')

In [24]:
y_test[2734]

2

In [25]:
len(y_pred_df)

3000

In [26]:
#episodes that are misdiagnosed
non_success_df = y_pred_df[y_pred_df['y_pred']!= y_pred_df['y_actual']]
non_success_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
2,1.0,2.0,0.0,-1.0,0.0,[B],0.0,1.0
4,1.0,4.0,0.0,-1.0,0.0,[B],2.0,1.0
8,1.0,8.0,0.0,-1.0,0.0,[B],0.0,1.0
9,1.0,9.0,0.0,-1.0,0.0,[B],2.0,1.0
10,1.0,10.0,0.0,-1.0,0.0,[B],0.0,1.0
...,...,...,...,...,...,...,...,...
2993,1.0,2993.0,0.0,-1.0,0.0,[B],0.0,1.0
2994,1.0,2994.0,0.0,-1.0,0.0,[B],2.0,1.0
2997,1.0,2997.0,0.0,-1.0,0.0,[B],0.0,1.0
2998,1.0,2998.0,0.0,-1.0,0.0,[B],2.0,1.0


In [27]:
non_success_df.index

Int64Index([   2,    4,    8,    9,   10,   12,   13,   14,   15,   20,
            ...
            2980, 2981, 2982, 2985, 2990, 2993, 2994, 2997, 2998, 2999],
           dtype='int64', length=1597)

In [28]:
b_non_succ_df = non_success_df[non_success_df.y_pred==1]
b_non_succ_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
2,1.0,2.0,0.0,-1.0,0.0,[B],0.0,1.0
4,1.0,4.0,0.0,-1.0,0.0,[B],2.0,1.0
8,1.0,8.0,0.0,-1.0,0.0,[B],0.0,1.0
9,1.0,9.0,0.0,-1.0,0.0,[B],2.0,1.0
10,1.0,10.0,0.0,-1.0,0.0,[B],0.0,1.0
...,...,...,...,...,...,...,...,...
2993,1.0,2993.0,0.0,-1.0,0.0,[B],0.0,1.0
2994,1.0,2994.0,0.0,-1.0,0.0,[B],2.0,1.0
2997,1.0,2997.0,0.0,-1.0,0.0,[B],0.0,1.0
2998,1.0,2998.0,0.0,-1.0,0.0,[B],2.0,1.0


In [29]:
c_non_succ_df = non_success_df[non_success_df.y_pred==2]
c_non_succ_df

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
